# Introduction to Plan


### Context 

> Warning: This is still experimental and may change during May / June 2019

We introduce here an object which is crucial to scale to industrial Federated Learning: the Plan. It reduces dramatically the bandwidth usage, allows asynchronous schemes and give more autonomy to remote devices. The original concept of plan can be found in the paper [Towards Federated Learning at Scale: System Design](https://arxiv.org/pdf/1902.01046.pdf), but it has been adapted to our needs in the PySyft library using PyTorch.

A Plan is intended to store a sequence of torch operations, just like a function, but it allows to send this sequence of operations to remote workers and to keep a reference to it. This way, to compute remotely this sequence of $n$ operations on some remote input referenced through pointers, instead of sending $n$ messages you need now to send a single message with the references of the plan and the pointers. Actually, it's so much like a function that you need a function to build a plan! Hence, for high level users, the notion of plan disappears and is replaced by a magic feature which allow to send to remote workers arbitrary functions containing sequential torch functions.

One thing to notice is that the class of functions that you can transform into plans is currently limited to sequences of hooked torch operations exclusively. This excludes in particular logical structures like `if`, `for` and `while` statements, even if we are working to have workarounds soon. _To be completely precise, you can use these but the logical path you take (first `if` to False and 5 loops in `for` for example) in the first computation of your plan will be the one kept for all the next computations, which we want to avoid in the majority of cases._

Authors:
- Théo Ryffel - Twitter [@theoryffel](https://twitter.com/theoryffel) - GitHub: [@LaRiffle](https://github.com/LaRiffle)
	

### Imports and model specifications

First let's make the official imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

And than those specific to PySyft

In [2]:
import syft as sy  # import the Pysyft library
hook = sy.TorchHook(torch)  # hook PyTorch ie add extra functionalities 
server = hook.local_worker

We define remote workers or _devices_, to be consistent with the notions provided in the reference article.
We provide them with some data.

In [3]:
x11 = torch.tensor([-1, 2.]).tag('input_data')
x12 = torch.tensor([1, -2.]).tag('input_data2')
x21 = torch.tensor([-1, 2.]).tag('input_data')
x22 = torch.tensor([1, -2.]).tag('input_data2')

device_1 = sy.VirtualWorker(hook, id="device_1", data=(x11, x12)) 
device_2 = sy.VirtualWorker(hook, id="device_2", data=(x21, x22))
devices = device_1, device_2

### Basic example

Let's define a function that we want to transform into a plan. To do so, it's as simple as adding a decorator above the function definition!

In [4]:
@sy.func2plan
def plan_double_abs(x):
    x = x + x
    x = torch.abs(x)
    return x

Let's check, yes we have now a plan!

In [5]:
plan_double_abs

<Plan plan_double_abs id:5123896729 owner:me>

To use a plan, you need two things: to build the plan (_ie register the sequence of operations present in the function_) and to send it to a worker / device. Fortunately you can do this very easily!

We first get a reference to some remote data: a request is sent over the network and a reference pointer is returned.

In [6]:
pointer_to_data = device_1.search('input_data')[0]
pointer_to_data

(Wrapper)>[PointerTensor | me:28752364492 -> device_1:4927566757]
	Tags: input_data 
	Shape: torch.Size([2])

We tell the plan it must be executed remotely on the device, but actually nothing happens on the network because we have not provided any input data! You can now observe that there is an attribute location specified `location:device_1`.

In [7]:
plan_double_abs.send(device_1)

<Plan plan_double_abs id:5123896729 owner:me location:device_1>

One important thing to remember is now that we pre-set ahead of computation the id(s) where the result(s) should be stored. This will allow to send commands asynchronously, to already have a reference to a virtual result and to continue local computations without waiting for the remote result to be computed. One major application is when you require computation of a batch on device_1 and don't want to wait for this computation to end to launch another batch computation on device_2.

We now feed the plan with a reference pointer to some data. Three things happens: (1) the plan is built, (2) it is sent to the device and (3) it is run remotely.
1. All the commands are executed sequentially by the local worker, and instead of being sent to the device they are catched by the plan and stored in his ... plan attribute!
2. This newly plan object is sent to the remote worker in a single communication round.
3. An other command is issued to run this plan remotely, so that the predefined location of the output of the plan now contains the result (remember we pre-set location of result ahead of computation). This also require a single communication round. _That's the place where we could run asynchronously_.

The result is simply a pointer, just like when you call an usual hooked torch function!

In [8]:
%%time
# %%time is a Magic comand to log a cell's execution time

pointer_to_result = plan_double_abs(pointer_to_data)
print(pointer_to_result)

[PointerTensor | me:14841324516 -> device_1:8459055087]
CPU times: user 52.8 ms, sys: 3.89 ms, total: 56.7 ms
Wall time: 54.7 ms


And you can simply ask the value back.

In [9]:
pointer_to_result.get()

tensor([2., 4.])

What happen now if you ask a second computation with this plan?

In [10]:
pointer_to_data = device_1.search('input_data2')[0]

Now it is much faster because there is a single communication round: we already have a reference to the remote plan, so we can require a remote execution and just provide the reference location of the new remote inputs. For the end user, nothing changes.

In [11]:
%time
pointer_to_result = plan_double_abs(pointer_to_data)
print(pointer_to_result.get())

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs
tensor([2., 4.])


### Towards a concrete example

But what we want to do is to apply Plan to Deep and Federated Learning, right? So let's look to a slightly more complicated example, using neural networks as you might be willing to use them.
Note that we are now transforming a method into a plan, so we use the `@` `sy.meth2plan` decorator instead

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    @sy.method2plan
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)


In [14]:
net = Net()

So the only thing we did is adding the sy.meth2plan decorator! And check that `net.forward` is again a plan.

In [15]:
net.forward

<Plan forward id:2465536292 owner:me>

Now there is a subtlety: because the plan depend on the net instance, if you send the plan *you also need to send the model*.

> For developers: this is not compulsory as you actually have a reference to the model in the plan, we could call model.send internally.

In [16]:
net.send(device_1)

Net(
  (fc1): Linear(in_features=2, out_features=3, bias=True)
  (fc2): Linear(in_features=3, out_features=2, bias=True)
)

In [17]:
net.forward.send(device_1)

<Plan forward id:2465536292 owner:me location:device_1>

Let's retrieve some remote data

In [18]:
pointer_to_data = device_1.search('input_data')[0]

Then, the syntax is just like normal remote sequential execution, that is, just like local execution. But compared to classic remote execution, there is a single communication round for each execution (except the first time where, as described above, we first build and send the plan).

In [19]:
pointer_to_result = net.forward(pointer_to_data)
pointer_to_result

[PointerTensor | me:44679384496 -> device_1:4514710581]

And we get the result as usual!

In [20]:
pointer_to_result.get()

tensor([-0.6341, -0.7559], requires_grad=True)

Et voilà! We have seen how to dramatically reduce the communication between the local worker (or server) and the remote devices!

### Switch between workers

One major feature that we want to have is to use the same plan for several workers, that we would change depending on the remote batch of data we are considering.
In particular, we don't want to rebuild the plan each time we change of worker. Let's see how we do this, using the previous example with our small network.

In [22]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    @sy.method2plan
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

In [23]:
net = Net()

Here are the main steps we just executed

In [24]:
net.send(device_1)
net.forward.send(device_1)
pointer_to_data = device_1.search('input_data')[0]
pointer_to_result = net(pointer_to_data)
pointer_to_result.get()

tensor([-0.8864, -0.5312], requires_grad=True)

Let's get the model and the network back

In [25]:
net.get()
net.forward.get()

<Plan forward id:1260222784 owner:me built>

And actually the syntax is straight forward: we just send it to another device

In [26]:
net.send(device_2)
net.forward.send(device_2)
pointer_to_data = device_2.search('input_data')[0]
pointer_to_result = net(pointer_to_data)
pointer_to_result.get()

tensor([-0.8864, -0.5312], requires_grad=True)

At this point, the interest of Plans might not be straightforward, but we will soon reuse all these elements for a larger scale Federated Learning task where we will need more complex interactions between workers.

### Star PySyft on GitHub

The easiest way to help our community is just by starring the repositories! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Pick our tutorials on GitHub!

We made really nice tutorials to get a better understanding of what Federated and Privacy-Preserving Learning should look like and how we are building the bricks for this to happen.

- [Checkout the PySyft tutorials](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! 

- [Join slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! If you want to start "one off" mini-projects, you can go to PySyft GitHub Issues page and search for issues marked `Good First Issue`.

- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

- [Donate through OpenMined's Open Collective Page](https://opencollective.com/openmined)